In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as random
import math
import itertools
import operator
import helpFunctions as help
import helpFunctionsRL as helpRL

In [2]:
N = 3
B = 10
alpha = 0.85
beta = 1
p_exp_zero = 0.999
episodes = 10**4
Sum_Rmin_list = [4,8,12,16,20]

In [3]:
from operator import index


sr_rl_min = []

for m in Sum_Rmin_list:

    sr = []
    p_exp = [0 for i in range(0,episodes)]
    p_exp[0] = p_exp_zero
    
    allPossibleStates = helpRL.everyState(N,m)
    allPossibleActions = helpRL.everyAction(B,N)
    qTable = np.zeros([len(allPossibleStates),len(allPossibleActions)])


    for i in range(0,episodes):
        actions = list(helpRL.allStates(B,N))
        queue = help.generate_queue(m,N)
        currentState = queue[:,0]

        check = helpRL.checkIfAllNegative(currentState)
        time = 0
        p_exp[i] = max(p_exp[i-1]**i,0.01)
        M = N

        while check:
            #currentActionIndex = random.randint(0,len(actions)-1)
            currentActionIndex = helpRL.actionByAgent(qTable,p_exp[i],currentState,allPossibleStates,allPossibleActions,actions)

            rewardAction = np.array(actions[currentActionIndex])
            nextState = np.array(currentState) - rewardAction
            nextState = tuple([int(elem) for elem in nextState])
            #print("Current State: ",currentState,"Action take: ",rewardAction,"Next State: ",nextState)

            check2 = helpRL.checkIfOneNegative(nextState)
            if check2[0]:
                [nextState,queue] = helpRL.deleteElement(queue,nextState,check2[1],time)
                M = M-len(check2[1]) 
                if M==0:
                    break
                actions = list(helpRL.allStates(B,M))
            
            check = helpRL.checkIfAllNegative(nextState)
            if check:
                update = helpRL.reward(currentState, rewardAction)
                xCoord = allPossibleStates.index(tuple(nextState))
                qTable[xCoord, currentActionIndex] +=  beta*update

            currentState = nextState

            time = time+1
        
        sr.append(sum(queue[:,3])/N)
    sr_rl = sum(sr)/episodes
    sr_rl_min.append(sr_rl)
